In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn


In [ ]:
import urllib.request
urllib.request.urlretrieve('ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt', 'stations.txt')

In [ ]:
[line for line in open('stations.txt','r')][:10]

In [ ]:
stations = {}

for line in open('stations.txt','r'):
    if 'GSN' in line:
        fields = line.split()
    
        stations[fields[0]] = ' '.join(fields[4:])

In [ ]:
len(stations)

In [ ]:
def find_station(s):
    found = {code: name for code, name in stations.items() if s in name}
    
    return found

In [ ]:
find_station('LIHUE')

In [ ]:
url = 'https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/gsn/USW00022536.dly'
urllib.request.urlretrieve(url,'USW00022536.dly')

In [ ]:
open('USW00022536.dly','r').readlines()[:10]

In [ ]:
def parsefile(filename):
    return np.genfromtxt(filename,
                        delimiter = dly_delimiter,
                        usecols = dly_usecols,
                        dtype = dly_dtype,
                        names = dly_names)

In [ ]:
dly_delimiter = [11, 4, 2, 4] + [5, 1, 1, 1] * 31
dly_usecols = [1, 2, 3] + [4*i for i in range(1, 32)]
dly_dtype = [np.int32, np.int32, (np.str_, 4)] + [np.int32]*31
dly_names = ['year','month', 'obs'] + [str(i) for i in range(1,32)]

In [ ]:
lihue = parsefile('USW00022536.dly')

In [ ]:
def unroll(record):
    startdate = np.datetime64('{}-{:02}'.format(record['year'], record['month']))
    dates = np.arange(startdate, startdate + np.timedelta64(1, 'M'), np.timedelta64(1, 'D'))
    
    rows = [(date, record[str(i+1)]/10) for i, date in enumerate(dates)]
    
    #return np.array(rows)
    return np.array(rows, dtype=[('date', 'M8[D]'), ('value', 'd')])

In [ ]:
def get_obs(fname, obs):
    return np.concatenate([unroll(row) for row in parsefile(fname) if row[2] == obs])

In [ ]:
tmax = get_obs('USW00022536.dly', 'TMAX')
tmax[:10]